# Setup

In [ ]:
!nvidia-smi

Thu Mar 25 16:04:50 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.56       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install --upgrade fastai torchmetrics -qqq

     |████████████████████████████████| 194kB 8.3MB/s 
     |████████████████████████████████| 184kB 27.5MB/s 
     |████████████████████████████████| 776.8MB 21kB/s 
     |████████████████████████████████| 61kB 8.3MB/s 
     |████████████████████████████████| 12.8MB 244kB/s 
ERROR: torchtext 0.9.0 has requirement torch==1.8.0, but you'll have torch 1.7.1 which is incompatible.


Get the data

In [ ]:
!wget "https://data.cityofchicago.org/api/views/ijzp-q8t2/rows.csv"

--2021-03-25 16:08:30--  https://data.cityofchicago.org/api/views/ijzp-q8t2/rows.csv
Resolving data.cityofchicago.org (data.cityofchicago.org)... 52.206.140.199, 52.206.68.26, 52.206.140.205
Connecting to data.cityofchicago.org (data.cityofchicago.org)|52.206.140.199|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/csv]
Saving to: ‘rows.csv’

rows.csv                [               <=>  ]   1.60G  2.82MB/s    in 9m 3s   

2021-03-25 16:17:33 (3.02 MB/s) - ‘rows.csv’ saved [1721603444]



In [ ]:
from collections import Counter, defaultdict
from typing import Dict, List

import torch
from fastai.tabular.all import *
from torchmetrics import F1, Accuracy, Precision, Recall
from torch.nn.utils.convert_parameters import parameters_to_vector, vector_to_parameters
from torch.optim import Adam, Optimizer

import pandas as pd
from tqdm.notebook import tqdm

import gc

# Data preprocessing

Read the data and drop the unnecessary features (columns)

In [ ]:
data = pd.read_csv("./rows.csv")
data.drop(
    [
        "ID",
        "Case Number",
        "IUCR",
        "Description",
        "Arrest",
        "FBI Code",
        "Latitude",
        "Longitude",
        "Location",
        "Updated On",
        "Date",
        "X Coordinate",
        "Y Coordinate",
        "Domestic",
    ],
    axis=1,
    inplace=True,
)

data

,Block,Primary Type,Location Description,Beat,District,Ward,Community Area,Year
0,043XX S WOOD ST,BATTERY,RESIDENCE,924,9.0,12.0,61.0,2015
1,008XX N CENTRAL AVE,THEFT,CTA BUS,1511,15.0,29.0,25.0,2015
2,082XX S INGLESIDE AVE,THEFT,RESIDENCE,631,6.0,8.0,44.0,2018
3,035XX W BARRY AVE,NARCOTICS,SIDEWALK,1412,14.0,35.0,21.0,2015
4,0000X N LARAMIE AVE,ASSAULT,APARTMENT,1522,15.0,28.0,25.0,2015
...,...,...,...,...,...,...,...,...
7296597,057XX S PRINCETON AVE,DECEPTIVE PRACTICE,RESIDENCE,711,7.0,20.0,68.0,2020
7296598,080XX S INDIANA AVE,THEFT,RESIDENCE,623,6.0,6.0,44.0,2020
7296599,079XX S ELLIS AVE,OTHER OFFENSE,APARTMENT,624,6.0,8.0,44.0,2021
7296600,010XX W TAYLOR ST,NARCOTICS,STREET,1232,12.0,11.0,28.0,2021


For reproducibility purposes, we will only consider cases that happened in 2020 or earlier

In [ ]:
data.drop(data[data["Year"] > 2020].index, inplace=True)
data.reset_index(drop=True, inplace=True)

In [ ]:
len(data)

7260345

## Missing values

Percentage of missing values in each column

In [ ]:
data.isnull().sum() / len(data) * 100

Block                   0.000000
Primary Type            0.000000
Location Description    0.109155
Beat                    0.000000
District                0.000647
Ward                    8.468372
Community Area          8.449791
Year                    0.000000
dtype: float64

In [ ]:
# Location Description already has a value called `OTHER` so we'll use it

data["Location Description"].fillna("OTHER", inplace=True)

In [ ]:
# Fill missing values with the most common value

most_common = data["District"].mode().values[0]             # most common value
data["District"].fillna(value=most_common, inplace=True)    

data["District"] = data["District"].astype(int)             # District should be an integer

The columns **Ward** and **Community Area** have about **600K** lines with missing values, so we cant' just replace them with the most common value.

Instead, we'll make use of the observation that most the features in our dataset are spatial features, furthermore they are heirarchical features.

For example, **Block** is the *smallest* spatial feature.

**Beat**: Three to five beats make up a police sector, and three sectors make up a police **District**.

**Block**s can also be organized into **Ward**s and **Community Area**s.

So, if a block is seen once in the dataset with a specific ward or community area, then it makes sense that the block and its corresponding ward (or community area) will always come together.

The same applies for Beat and District.

This function implements of the above observation

In [ ]:
def impute(df, known_columns: List[str], unknown_col: str) -> Dict:
    """Fills missing values in `unknown_col` by using information from `known_columns`

    Args:
        df (DataFrame): the data
        known_columns (List[str]): columns that don't have any missing values
        unknown_col (str): name of the column that has missing values

    Returns:
        Dict: a dictionary that contains found missing values, keys are rows indices and values are the found values
    """
    found = dict()

    for known_col in known_columns:
        known2unknown = defaultdict(Counter)

        for known_el, unknown_el in zip(df[known_col], df[unknown_col]):
            if pd.notnull(unknown_el): # if it's not a missing value
                known2unknown[known_el].update([unknown_el])

        for idx, (known_el, unknown_el) in enumerate(zip(df[known_col], df[unknown_col])):
            if pd.isnull(unknown_el) and known_el in known2unknown:
                found[idx] = known2unknown[known_el].most_common(n=1)[0][0] # take the most common value associated with `known_el`

    return found

In [ ]:
# impute the missing values in Ward based on the values in Block, Beat and District

imputed_ward = impute(data, ["Block", "Beat", "District"], "Ward")

In [ ]:
data.loc[imputed_ward.keys(), "Ward"] = list(imputed_ward.values())

In [ ]:
# check the number of new missing values
data["Ward"].isnull().sum()

0

Repeat the same process for **Community Area**

In [ ]:
imputed_comm_area = impute(data, ["Block", "Beat", "District"], "Community Area")

In [ ]:
data.loc[imputed_comm_area.keys(), "Community Area"] = list(imputed_comm_area.values())

In [ ]:
data["Community Area"].isnull().sum()

0

No more missing values are left in the dataset

In [ ]:
data.isnull().sum() / len(data) * 100

Block                   0.0
Primary Type            0.0
Location Description    0.0
Beat                    0.0
District                0.0
Ward                    0.0
Community Area          0.0
Year                    0.0
dtype: float64

In [ ]:
# these should be integers
data["Ward"] = data["Ward"].astype(int)
data["Community Area"] = data["Community Area"].astype(int)

## Feature Engineering

In this section, we will reduce the number of types and group them into smaller number of categories

Here we check how many type we have

In [ ]:
data['Primary Type'].value_counts()

THEFT                                1531023
BATTERY                              1329392
CRIMINAL DAMAGE                       826525
NARCOTICS                             735909
ASSAULT                               459926
OTHER OFFENSE                         450336
BURGLARY                              407958
MOTOR VEHICLE THEFT                   334507
DECEPTIVE PRACTICE                    305004
ROBBERY                               272795
CRIMINAL TRESPASS                     205226
WEAPONS VIOLATION                      86148
PROSTITUTION                           69371
PUBLIC PEACE VIOLATION                 50770
OFFENSE INVOLVING CHILDREN             50529
CRIM SEXUAL ASSAULT                    27944
SEX OFFENSE                            27760
INTERFERENCE WITH PUBLIC OFFICER       17499
GAMBLING                               14594
LIQUOR LAW VIOLATION                   14459
ARSON                                  12165
HOMICIDE                               10835
KIDNAPPING

Here we create a dictionary to categorize the crime types into smaller categories as all of the types in the dictionary's values will be replaced by the corresponding key value.

In [ ]:
categories = {
    "forbidden practices": ["narcotics", "prostitution", "gambling", "obscenity", "narcotic violation"],
    "theft": ["burglary", "deceptive practice", "motor vehicle theft", "robbery"],
    "assault": [
        "crime sexual assault",
        "offense involving children",
        "sex offense",
        "homicide",
        "human trafficking",
        "criminal sexual assault",
        "assault",
    ],
    "public peace violation": [
        "weapons violation",
        "criminal defacement",
        "criminal trespass",
        "arson",
        "kidnapping",
        "stalking",
        "intimidation",
        "public indecency",
    ],
}

for category in categories:
    data["Primary Type"].replace([x.upper() for x in categories[category]], category.upper(), inplace=True)

Now we check the new types in the dataset

In [ ]:
data['Primary Type'].value_counts()

THEFT                                2851287
BATTERY                              1329392
CRIMINAL DAMAGE                       826525
FORBIDDEN PRACTICES                   820575
ASSAULT                               551687
OTHER OFFENSE                         450336
PUBLIC PEACE VIOLATION                369591
CRIM SEXUAL ASSAULT                    27944
INTERFERENCE WITH PUBLIC OFFICER       17499
LIQUOR LAW VIOLATION                   14459
CONCEALED CARRY LICENSE VIOLATION        668
NON-CRIMINAL                             172
OTHER NARCOTIC VIOLATION                 138
NON - CRIMINAL                            38
RITUALISM                                 24
NON-CRIMINAL (SUBJECT SPECIFIED)           9
DOMESTIC VIOLENCE                          1
Name: Primary Type, dtype: int64

## Final dataset

In [ ]:
data

,Block,Primary Type,Location Description,Beat,District,Ward,Community Area,Year
0,043XX S WOOD ST,BATTERY,RESIDENCE,924,9,12,61,2015
1,008XX N CENTRAL AVE,THEFT,CTA BUS,1511,15,29,25,2015
2,082XX S INGLESIDE AVE,THEFT,RESIDENCE,631,6,8,44,2018
3,035XX W BARRY AVE,FORBIDDEN PRACTICES,SIDEWALK,1412,14,35,21,2015
4,0000X N LARAMIE AVE,ASSAULT,APARTMENT,1522,15,28,25,2015
...,...,...,...,...,...,...,...,...
7260340,046XX W BELMONT AVE,THEFT,STREET,2521,25,31,19,2020
7260341,022XX W WARREN BLVD,ASSAULT,APARTMENT,1223,12,27,28,2020
7260342,062XX S ALBANY AVE,ASSAULT,RESIDENCE,823,8,16,66,2020
7260343,057XX S PRINCETON AVE,THEFT,RESIDENCE,711,7,20,68,2020


In [ ]:
data.to_feather("processed.feather")

# Modeling

In [ ]:
def set_seed(seed=0):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

set_seed()

## Create train/validation datasets and dataloaders

In [ ]:
splits = RandomSplitter(valid_pct=0.05, seed=0)(range_of(data))

prepared_df = TabularPandas(
    data,
    cat_names=['Block', 'Location Description', 'Beat', 'District', 'Ward', 'Community Area', 'Year'], # categorical variables
    y_names="Primary Type",
    procs=[Categorify], # converts categorical variables to integers under the hood
    splits=splits,
)

dataloaders = prepared_df.dataloaders(bs=64) # batch size = 64

In [ ]:
dataloaders.show_batch()

,Block,Location Description,Beat,District,Ward,Community Area,Year,Primary Type
0,023XX W BELDEN AVE,RESIDENCE,1432,14,32,22,2019,THEFT
1,026XX S INDIANA AVE,APARTMENT,2112,1,2,35,2004,BATTERY
2,067XX W BELMONT AV,AIRPORT/AIRCRAFT,1632,16,41,15,2001,THEFT
3,078XX S CORNELL AVE,RESIDENCE,411,4,8,43,2020,THEFT
4,022XX S ARCHER AVE,GROCERY FOOD STORE,923,9,25,34,2007,BATTERY
5,046XX S HALSTED ST,GROCERY FOOD STORE,921,9,11,61,2005,THEFT
6,004XX E 63RD ST,GROCERY FOOD STORE,313,3,20,42,2003,LIQUOR LAW VIOLATION
7,021XX W POTOMAC AVE,SIDEWALK,1424,14,32,24,2008,ASSAULT
8,013XX W TAYLOR ST,APARTMENT,1231,12,28,28,2020,THEFT
9,019XX N KOSTNER AVE,APARTMENT,2533,25,31,20,2002,BATTERY


## Create model architecture

This will calculate the appropriate embedding dimension for each column based on the number of unique values in it and a tested rule of thumb:

```python
embedding_dim = min(600, round(1.6 * num_unique_values ** 0.56))
```

In [ ]:
embeddings_sizes = get_emb_sz(dataloaders.train_ds, {"Block": 128}) # give the `Block` column a default dimension of 128 (instead of 600)
embeddings_sizes

[(61385, 128), (215, 32), (305, 39), (25, 10), (51, 14), (79, 18), (21, 9)]

In [ ]:
# Number of output classes (should be 17)
num_classes = dataloaders.c
num_classes

17

Use `fastai`'s `TabularModel` to create an entity embeddings model

In [ ]:
def create_model():
    model = TabularModel(
        emb_szs=embeddings_sizes,       # sizes of the embeddings matrices
        out_sz=num_classes,             # Number of output classes
        layers=[128, 64],               # linear (dense) layers sizes
        embed_p=0.1,                    # dropout probability
        n_cont=0,                       # number of continuous variables (0)
        bn_cont=False,                  # we don't use a BatchNorm layer on the continuous variables since we don't have any
    ).cuda()

    return model

The model's architecture

In [ ]:
create_model()

TabularModel(
  (embeds): ModuleList(
    (0): Embedding(61385, 128)
    (1): Embedding(215, 32)
    (2): Embedding(305, 39)
    (3): Embedding(25, 10)
    (4): Embedding(51, 14)
    (5): Embedding(79, 18)
    (6): Embedding(21, 9)
  )
  (emb_drop): Dropout(p=0.1, inplace=False)
  (layers): Sequential(
    (0): LinBnDrop(
      (0): BatchNorm1d(250, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (1): Linear(in_features=250, out_features=128, bias=False)
      (2): ReLU(inplace=True)
    )
    (1): LinBnDrop(
      (0): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (1): Linear(in_features=128, out_features=64, bias=False)
      (2): ReLU(inplace=True)
    )
    (2): LinBnDrop(
      (0): Linear(in_features=64, out_features=17, bias=True)
    )
  )
)

## Evaluation metrics

In [ ]:
# metrics
accuracy = Accuracy().cuda()

precision_macro = Precision(num_classes=num_classes, average="macro").cuda()
recall_macro = Recall(num_classes=num_classes, average="macro").cuda()
f1_macro = F1(num_classes=num_classes, average="macro").cuda()

In [ ]:
loss_fn = nn.CrossEntropyLoss().cuda()

In [ ]:
@torch.no_grad()
def evaluate(model):
    model.eval()
    eval_loss = 0.0
    for x, _, y in tqdm(dataloaders.valid):
        x, y = x.cuda(), y.cuda()

        logits = model(x)
        y_pred = logits.argmax(dim=1).view(-1, 1)
        
        eval_loss += loss_fn(logits, y.view(-1).long()).item()

        # add current predictions to the metrics' stack
        accuracy.update(y_pred, y)        
        precision_macro.update(y_pred, y)
        recall_macro.update(y_pred, y)
        f1_macro.update(y_pred, y)

    eval_loss /= len(dataloaders.valid)

    print(f"loss = {eval_loss}")
    print(f"accuracy = {accuracy.compute() * 100}")
    print(f"precision_macro = {precision_macro.compute() * 100}")
    print(f"recall_macro = {recall_macro.compute() * 100}")
    print(f"f1_macro = {f1_macro.compute() * 100}")

# Optimization Algorithms

## Quickprop

In [ ]:
class Quickprop(Optimizer):
    """Implements the Quickprop optimization algorithm as described in
    `An Empirical Study of Learning Speed in Back-Propagation Networks` (Fahlman, 1988) (Section 3.4)

    Args:
        params (iterable): iterable of parameters to optimize or dicts defining
            parameter groups
        lr (float, optional): learning rate (default: 1)
        sgd_lr (float, optional): learning rate of stochastic gradient descent (check comments for more details). Defaults to 1e-3.
        mu (float, optional): Maximum growth factor. Defaults to 1.75.
    """
    def __init__(self, params, lr=1.0, sgd_lr=1e-3, mu=1.75):
        defaults = dict(lr=lr)
        super().__init__(params, defaults)
        self.sgd_lr = sgd_lr
        self.mu = mu  # maximum growth factor
        self.state["step"] = 0

    @torch.no_grad()
    def first_step(self, closure):
        with torch.enable_grad():
            loss = closure()

        for group in self.param_groups:
            for p in group["params"]:
                if p.grad is None:
                    continue
                delta = -p.grad * self.sgd_lr
                p.add_(delta)
                self.state[p].update({"prev_grad": p.grad.clone(), "prev_delta": delta.clone()})

        return loss

    @torch.no_grad()
    def step(self, closure):
        # first step: bootstrap the prev_ values using an sgd step
        if self.state["step"] == 0:
            return self.first_step(closure)

        with torch.enable_grad():
            loss = closure()

        for group in self.param_groups:
            lr = group["lr"]
            for p in group["params"]:
                if p.grad is None:
                    continue
                param_state = self.state[p]

                prev_grad = param_state["prev_grad"]
                curr_grad = p.grad

                prev_delta = param_state["prev_delta"]

                # if the previous grad is close to the current one, use sgd update rule instead (as suggested in the paper by Fahlman)
                selector = ((prev_grad - curr_grad).abs() < 1e-8).float()
                quickprop_factor = selector * (prev_delta / (prev_grad - curr_grad))
                sgd_factor = (1 - selector) * (-self.sgd_lr)

                """No weight step is allowed to be greater in magnitude than µ times the previous step for that weight"""
                quickprop_factor = quickprop_factor.clamp(-self.mu, self.mu)

                curr_delta = curr_grad * (quickprop_factor + sgd_factor)

                p.add_(curr_delta, alpha=lr)

                param_state.update({"prev_grad": curr_grad.clone(), "prev_delta": curr_delta.clone()})

        self.state["step"] += 1
        return loss

In [ ]:
model = create_model()
optimizer = Quickprop(model.parameters(), lr=1, sgd_lr=1e-2)

Train the model using the `Quickprop` optimizer

In [ ]:
%%time

num_epochs = 1 # one epoch is enough here

model.train()

for epoch in tqdm(range(num_epochs)):
    train_loss = 0.0
    for x, _, y in tqdm(dataloaders.train):

        def closure():
            if torch.is_grad_enabled():
                optimizer.zero_grad()
            logits = model(x.cuda())
            loss = loss_fn(logits, y.view(-1).long().cuda())
            if loss.requires_grad:
                loss.backward()
            return loss

        loss = optimizer.step(closure)
        train_loss += loss.item()
    
    train_loss /= len(dataloaders.train)
    print(f"epoch {epoch + 1} loss = {train_loss}")


epoch 1 loss = 1.5016131415596763

CPU times: user 11min 37s, sys: 53 s, total: 12min 30s
Wall time: 12min 48s


In [ ]:
evaluate(model)


loss = 1.4739087980175574
accuracy = 46.50483703613281
precision_macro = 21.49054718017578
recall_macro = 11.223831176757812
f1_macro = 10.38428020477295


In [ ]:
torch.save(model.state_dict(), "quickprop_model_state_dict.pt")

## Generalized Gauss-Newton

In [ ]:
class HessianFree(Optimizer):
    """
    Implements the Hessian-free algorithm presented in `Training Deep and
    Recurrent Networks with Hessian-Free Optimization`_ using the Generalized Gauss-Newton algorithm.

    Arguments:
        params (iterable): iterable of parameters to optimize or dicts defining
            parameter groups
        lr (float, optional): learning rate (default: 1)
        delta_decay (float, optional): Decay of the previous result of
            computing delta with conjugate gradient method for the
            initialization of the next conjugate gradient iteration
        damping (float, optional): Initial value of the Tikhonov damping
            coefficient. (default: 0.5)
        max_iter (int, optional): Maximum number of Conjugate-Gradient
            iterations (default: 50)

    .. _Training Deep and Recurrent Networks with Hessian-Free Optimization:
        https://doi.org/10.1007/978-3-642-35289-8_27
    """

    def __init__(self, params, lr=1, damping=0.5, delta_decay=0.95, cg_max_iter=100):

        if not (0.0 < lr <= 1):
            raise ValueError("Invalid lr: {}".format(lr))

        if not (0.0 < damping <= 1):
            raise ValueError("Invalid damping: {}".format(damping))

        if not cg_max_iter > 0:
            raise ValueError("Invalid cg_max_iter: {}".format(cg_max_iter))

        defaults = dict(
            alpha=lr,
            damping=damping,
            delta_decay=delta_decay,
            cg_max_iter=cg_max_iter,
        )
        super(HessianFree, self).__init__(params, defaults)

        if len(self.param_groups) != 1:
            raise ValueError("HessianFree doesn't support per-parameter options (parameter groups)")

        self._params = self.param_groups[0]["params"]

    def _gather_flat_grad(self):
        views = list()
        for p in self._params:
            if p.grad is None:
                view = p.data.new(p.data.numel()).zero_()
            elif p.grad.data.is_sparse:
                view = p.grad.data.to_dense().view(-1)
            else:
                view = p.grad.contiguous().view(-1)
            views.append(view)
        return torch.cat(views, 0)

    def step(self, closure, b=None):
        """
        Performs a single optimization step.

        Arguments:
            closure (callable): A closure that re-evaluates the model
                and returns a tuple of the loss and the output.
            b (callable, optional): A closure that calculates the vector b in
                the minimization problem x^T . A . x + x^T b.
        """
        assert len(self.param_groups) == 1

        group = self.param_groups[0]
        alpha = group["alpha"]
        delta_decay = group["delta_decay"]
        cg_max_iter = group["cg_max_iter"]
        damping = group["damping"]

        state = self.state[self._params[0]]
        state.setdefault("func_evals", 0)
        state.setdefault("n_iter", 0)

        loss_before, output = closure()
        current_evals = 1
        state["func_evals"] += 1

        # Gather current parameters and respective gradients
        flat_params = parameters_to_vector(self._params)
        flat_grad = self._gather_flat_grad()

        # Define linear operator
        # Generalized Gauss-Newton vector product
        def A(x):
            return self._Gv(loss_before, output, x, damping)

        M = None

        b = flat_grad.detach() if b is None else b().detach().flatten()

        # Initializing Conjugate-Gradient (Section 20.10)
        if state.get("init_delta") is not None:
            init_delta = delta_decay * state.get("init_delta")
        else:
            init_delta = torch.zeros_like(flat_params)

        eps = torch.finfo(b.dtype).eps

        # Conjugate-Gradient
        deltas, Ms = self._CG(
            A=A, b=b.neg(), x0=init_delta, M=M, max_iter=cg_max_iter, tol=1e1 * eps, eps=eps, martens=True
        )

        # Update parameters
        delta = state["init_delta"] = deltas[-1]
        M = Ms[-1]

        vector_to_parameters(flat_params + delta, self._params)
        loss_now = closure()[0]
        current_evals += 1
        state["func_evals"] += 1

        # Conjugate-Gradient backtracking (Section 20.8.7)
        for (d, m) in zip(reversed(deltas[:-1][::2]), reversed(Ms[:-1][::2])):
            vector_to_parameters(flat_params + d, self._params)
            loss_prev = closure()[0]
            if float(loss_prev) > float(loss_now):
                break
            delta = d
            M = m
            loss_now = loss_prev

        # The Levenberg-Marquardt Heuristic (Section 20.8.5)
        reduction_ratio = (float(loss_now) - float(loss_before)) / M if M != 0 else 1

        if reduction_ratio < 0.25:
            group["damping"] *= 3 / 2
        elif reduction_ratio > 0.75:
            group["damping"] *= 2 / 3
        if reduction_ratio < 0:
            group["init_delta"] = 0

        # Line Searching (Section 20.8.8)
        beta = 0.8
        c = 1e-2
        min_improv = min(c * torch.dot(b, delta), 0)

        for _ in range(60):
            if float(loss_now) <= float(loss_before) + alpha * min_improv:
                break

            alpha *= beta
            vector_to_parameters(flat_params + alpha * delta, self._params)
            loss_now = closure()[0]
        else:  # No good update found
            alpha = 0.0
            loss_now = loss_before

        # Update the parameters (this time fo real)
        vector_to_parameters(flat_params + alpha * delta, self._params)

        return loss_now

    def _CG(self, A, b, x0, M=None, max_iter=50, tol=1.2e-6, eps=1.2e-7, martens=False):
        """
        Minimizes the linear system x^T.A.x - x^T b using the conjugate
            gradient method

        Arguments:
            A (callable): An abstract linear operator implementing the
                product A.x. A must represent a hermitian, positive definite
                matrix.
            b (torch.Tensor): The vector b.
            x0 (torch.Tensor): An initial guess for x.
            M (callable, optional): An abstract linear operator implementing
            the product of the preconditioner (for A) matrix with a vector.
            tol (float, optional): Tolerance for convergence.
            martens (bool, optional): Flag for Martens' convergence criterion.
        """

        x = [x0]
        r = A(x[0]) - b

        if M is not None:
            y = M(r)
            p = -y
        else:
            p = -r

        res_i_norm = r @ r

        if martens:
            m = [0.5 * (r - b) @ x0]

        for i in range(max_iter):
            Ap = A(p)

            alpha = res_i_norm / ((p @ Ap) + eps)

            x.append(x[i] + alpha * p)
            r = r + alpha * Ap

            if M is not None:
                y = M(r)
                res_ip1_norm = y @ r
            else:
                res_ip1_norm = r @ r

            beta = res_ip1_norm / (res_i_norm + eps)
            res_i_norm = res_ip1_norm

            # Martens' Relative Progress stopping condition (Section 20.4)
            if martens:
                m.append(0.5 * A(x[i + 1]) @ x[i + 1] - b @ x[i + 1])

                k = max(10, int(i / 10))
                if i > k:
                    stop = (m[i] - m[i - k]) / (m[i] + eps)
                    if stop < 1e-4:
                        break

            if res_i_norm < tol or torch.isnan(res_i_norm):
                break

            if M is not None:
                p = -y + beta * p
            else:
                p = -r + beta * p

        return (x, m) if martens else (x, None)

    def _Gv(self, loss, output, vec, damping):
        """
        Computes the generalized Gauss-Newton vector product.
        """
        Jv = self._Rop(output, self._params, vec)

        gradient = torch.autograd.grad(loss, output, create_graph=True)
        HJv = self._Rop(gradient, output, Jv)

        JHJv = torch.autograd.grad(output, self._params, grad_outputs=HJv.reshape_as(output), retain_graph=True)

        # Tikhonov damping (Section 20.8.1)
        return parameters_to_vector(JHJv).detach() + damping * vec

    @staticmethod
    def _Rop(y, x, v, create_graph=False):
        """
        Computes the product (dy_i/dx_j) v_j: R-operator
        """
        if isinstance(y, tuple):
            ws = [torch.zeros_like(y_i, requires_grad=True) for y_i in y]
        else:
            ws = torch.zeros_like(y, requires_grad=True)

        jacobian = torch.autograd.grad(y, x, grad_outputs=ws, create_graph=True)

        Jv = torch.autograd.grad(parameters_to_vector(jacobian), ws, grad_outputs=v, create_graph=create_graph)

        return parameters_to_vector(Jv)

In [ ]:
del model, optimizer
torch.cuda.empty_cache()
gc.collect();

In [ ]:
model = create_model()
optimizer = HessianFree(model.parameters(), lr=5e-3, cg_max_iter=3)

In [ ]:
%%time

num_epochs = 1

model.train()

for epoch in tqdm(range(num_epochs)):
    train_loss = 0.0
    for x, _, y in tqdm(dataloaders.train):

        def closure():
            if torch.is_grad_enabled():
                optimizer.zero_grad()
            logits = model(x.cuda())
            loss = loss_fn(logits, y.view(-1).long().cuda())
            if loss.requires_grad:
                loss.backward(create_graph=True) # set create_graph to True to enable calculating second order derivatives
            return loss, logits

        loss = optimizer.step(closure)
        train_loss += loss.item()

    train_loss /= len(dataloaders.train)
    print(f"epoch {epoch + 1} loss = {train_loss}")


epoch 1 loss = 0.9849135279130038

CPU times: user 1h 31min 33s, sys: 8min 19s, total: 1h 39min 53s
Wall time: 1h 42min 17s


In [ ]:
evaluate(model)


loss = 1.4738860429331422
accuracy = 46.4676513671875
precision_macro = 21.232929229736328
recall_macro = 11.131002426147461
f1_macro = 10.319028854370117


In [ ]:
torch.save(model.state_dict(), "gauss-newton_model_state_dict.pt")

## Delta-Bar-Delta

In [ ]:
class DeltaBarDelta(Optimizer):
    def __init__(self, params, lr_0=1e-4, k=1e-4, phi=0.1, theta=0.9):
        """Delta-Bar-Delta optimizer.

        Args:
            params: model parameters
            lr_0: initial learning rate. Defaults to 1e-4.
            k: value to add to learning rate. Defaults to 1e-4.
            phi: proportion to subtract from learning rate. Defaults to 0.1.
            theta: base of the exponential moving avergae of the gradient. Defaults to 0.9.
        """
        defaults = dict(lr_0=lr_0, k=k, phi=phi, theta=theta)
        super(DeltaBarDelta, self).__init__(params, defaults)
        self.k = k
        self.phi = phi
        self.theta = theta

    @torch.no_grad()
    def step(self, closure=None):
        loss = None

        if closure is not None:
            with torch.enable_grad():
                loss = closure()

        for group in self.param_groups:
            for p in group["params"]:
                if p.grad is None:
                    continue

                grad = p.grad

                state = self.state[p]
                if len(state) == 0:
                    state["lr"] = torch.ones_like(p) * group["lr_0"]
                    state["exp_avg"] = torch.zeros_like(p)  # exponential moving average of the gradient

                lr, exp_avg = state["lr"], state["exp_avg"]
                k, phi, theta = group["k"], group["phi"], group["theta"]

                # update learning rate
                sign = (exp_avg * grad).sign()
                delta_lr = torch.empty_like(lr)
                delta_lr[sign > 0] = k
                delta_lr[sign < 0] = -phi * lr[sign < 0]
                delta_lr[sign == 0] = 0
                lr += delta_lr

                # update the moving average
                exp_avg *= theta
                exp_avg += grad * (1.0 - theta)

                # update the parameter
                p -= lr * grad

        return loss

In [ ]:
del model, optimizer
torch.cuda.empty_cache()
gc.collect();

In [ ]:
model = create_model()
optimizer = DeltaBarDelta(model.parameters())

In [ ]:
%%time

num_epochs = 1

model.train()

for epoch in tqdm(range(num_epochs)):
    train_loss = 0.0
    for x, _, y in tqdm(dataloaders.train):
        optimizer.zero_grad()
        logits = model(x.cuda())
        loss = loss_fn(logits, y.view(-1).long().cuda())
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    train_loss /= len(dataloaders.train)
    print(f"epoch {epoch + 1} loss = {train_loss}")


epoch 1 loss = 1.5321878575057502

CPU times: user 30min 7s, sys: 5min 56s, total: 36min 4s
Wall time: 36min 43s


In [ ]:
evaluate(model)


loss = 1.499711928072586
accuracy = 46.288936614990234
precision_macro = 21.420398712158203
recall_macro = 11.014731407165527
f1_macro = 10.16046142578125


In [ ]:
torch.save(model.state_dict(), "delta-bar-delta_model_state_dict.pt")

## Backpropagation

In [ ]:
del model, optimizer
torch.cuda.empty_cache()
gc.collect();

In [ ]:
model = create_model()
optimizer = Adam(model.parameters())

In [ ]:
%%time

num_epochs = 1

model.train()

for epoch in tqdm(range(num_epochs)):
    train_loss = 0.0
    for x, _, y in tqdm(dataloaders.train):
        optimizer.zero_grad()
        logits = model(x.cuda())
        loss = loss_fn(logits, y.view(-1).long().cuda())
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    train_loss /= len(dataloaders.train)
    print(f"epoch {epoch + 1} loss = {train_loss}")


epoch 1 loss = 1.4882807564572809

CPU times: user 13min 2s, sys: 1min 46s, total: 14min 48s
Wall time: 15min 11s


In [ ]:
evaluate(model)


loss = 1.468169894059583
accuracy = 46.360050201416016
precision_macro = 21.5473690032959
recall_macro = 11.082728385925293
f1_macro = 10.268255233764648


In [ ]:
torch.save(model.state_dict(), "backprop_model_state_dict.pt")

# Results summary

In [ ]:
results = pd.DataFrame({
    "Training loss": [1.5016131415596763, 0.9849135279130038, 1.5321878575057502, 1.4882807564572809],
    "Validation loss": [1.4739087980175574, 1.4738860429331422, 1.499711928072586, 1.468169894059583],
    "Accuracy": [46.50483703613281, 46.4676513671875, 46.288936614990234, 46.360050201416016],
    "Precision (macro)": [21.49054718017578, 21.232929229736328, 21.420398712158203, 21.5473690032959],
    "Recall (macro)": [11.223831176757812, 11.131002426147461, 11.014731407165527, 11.082728385925293],
    "F1 (macro)": [10.38428020477295, 10.319028854370117, 10.16046142578125, 10.268255233764648],
}, index=["Quickprop", "Gauss-Newton", "Delta-Bar-Delta", "Backpropagation (Adam)"])

results

,Training loss,Validation loss,Accuracy,Precision (macro),Recall (macro),F1 (macro)
Quickprop,1.501613,1.473909,46.504837,21.490547,11.223831,10.384280
Gauss-Newton,0.984914,1.473886,46.467651,21.232929,11.131002,10.319029
Delta-Bar-Delta,1.532188,1.499712,46.288937,21.420399,11.014731,10.160461
Backpropagation (Adam),1.488281,1.468170,46.360050,21.547369,11.082728,10.268255


Quickprop and Adam seem to be the two best algorithms

In [ ]:
results.style.\
    highlight_max(subset=["Accuracy", "Precision (macro)", "Recall (macro)", "F1 (macro)"]).\
    highlight_min(subset=["Training loss", "Validation loss"]).\
    set_properties(**{"text-align": "center"})

,Training loss,Validation loss,Accuracy,Precision (macro),Recall (macro),F1 (macro)
Quickprop,1.501613,1.473909,46.504837,21.490547,11.223831,10.384280
Gauss-Newton,0.984914,1.473886,46.467651,21.232929,11.131002,10.319029
Delta-Bar-Delta,1.532188,1.499712,46.288937,21.420399,11.014731,10.160461
Backpropagation (Adam),1.488281,1.468170,46.360050,21.547369,11.082728,10.268255


# Experimenting with the model hyperparameters

## The new model

In [ ]:
embeddings_sizes = get_emb_sz(dataloaders.train_ds) # We will use bigger embeddings this time
embeddings_sizes

[(61385, 600), (215, 32), (305, 39), (25, 10), (51, 14), (79, 18), (21, 9)]

In [ ]:
# And more linear layers with more neurons
def create_model(embed_p=0.1):
    model = TabularModel(
        emb_szs=embeddings_sizes,       # sizes of the embeddings matrices
        out_sz=num_classes,             # Number of output classes
        layers=[256, 128, 64, 32],      # linear (dense) layers sizes
        embed_p=embed_p,                # dropout probability
        n_cont=0,                       # number of continuous variables (0)
        bn_cont=False,                  # we don't use a BatchNorm layer on the continuous variables since we don't have any
    ).cuda()

    return model

The new model architecture

In [ ]:
create_model()

TabularModel(
  (embeds): ModuleList(
    (0): Embedding(61385, 600)
    (1): Embedding(215, 32)
    (2): Embedding(305, 39)
    (3): Embedding(25, 10)
    (4): Embedding(51, 14)
    (5): Embedding(79, 18)
    (6): Embedding(21, 9)
  )
  (emb_drop): Dropout(p=0.1, inplace=False)
  (layers): Sequential(
    (0): LinBnDrop(
      (0): BatchNorm1d(722, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (1): Linear(in_features=722, out_features=256, bias=False)
      (2): ReLU(inplace=True)
    )
    (1): LinBnDrop(
      (0): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (1): Linear(in_features=256, out_features=128, bias=False)
      (2): ReLU(inplace=True)
    )
    (2): LinBnDrop(
      (0): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (1): Linear(in_features=128, out_features=64, bias=False)
      (2): ReLU(inplace=True)
    )
    (3): LinBnDrop(
      (0): BatchNorm1d(64, eps=1e-05, 

The best algorithms were Quickprop and Adam, so we'll use them to train the model here

## Using Quickprop

In [ ]:
model = create_model()
optimizer = Quickprop(model.parameters(), lr=1, sgd_lr=1e-2)

In [ ]:
%%time

num_epochs = 1

model.train()

for epoch in tqdm(range(num_epochs)):
    train_loss = 0.0
    for x, _, y in tqdm(dataloaders.train):

        def closure():
            if torch.is_grad_enabled():
                optimizer.zero_grad()
            logits = model(x.cuda())
            loss = loss_fn(logits, y.view(-1).long().cuda())
            if loss.requires_grad:
                loss.backward()
            return loss

        loss = optimizer.step(closure)
        train_loss += loss.item()
    
    train_loss /= len(dataloaders.train)
    print(f"epoch {epoch + 1} loss = {train_loss}")


epoch 1 loss = 1.5076262475752462

CPU times: user 15min 6s, sys: 2min 13s, total: 17min 20s
Wall time: 17min 44s


In [ ]:
evaluate(model)


loss = 1.476114185087983
accuracy = 46.387908935546875
precision_macro = 18.239896774291992
recall_macro = 11.25308895111084
f1_macro = 10.294276237487793


In [ ]:
torch.save(model.state_dict(), "quickprop_model_bigger_state_dict.pt")

### With higher embedding dropout probability

In [ ]:
model = create_model(embed_p=0.5)
optimizer = Quickprop(model.parameters(), lr=1, sgd_lr=1e-2)

In [ ]:
%%time

num_epochs = 1

model.train()

for epoch in tqdm(range(num_epochs)):
    train_loss = 0.0
    for x, _, y in tqdm(dataloaders.train):

        def closure():
            if torch.is_grad_enabled():
                optimizer.zero_grad()
            logits = model(x.cuda())
            loss = loss_fn(logits, y.view(-1).long().cuda())
            if loss.requires_grad:
                loss.backward()
            return loss

        loss = optimizer.step(closure)
        train_loss += loss.item()
    
    train_loss /= len(dataloaders.train)
    print(f"epoch {epoch + 1} loss = {train_loss}")


epoch 1 loss = 1.5297374556466579

CPU times: user 15min 18s, sys: 2min 14s, total: 17min 32s
Wall time: 17min 54s


In [ ]:
evaluate(model)


loss = 1.4881417199288982
accuracy = 46.29957580566406
precision_macro = 19.059648513793945
recall_macro = 11.078492164611816
f1_macro = 10.097859382629395


## Using Adam

In [ ]:
del model, optimizer
torch.cuda.empty_cache()
gc.collect();

In [ ]:
model = create_model()
optimizer = Adam(model.parameters())

In [ ]:
%%time

num_epochs = 1

model.train()

for epoch in tqdm(range(num_epochs)):
    train_loss = 0.0
    for x, _, y in tqdm(dataloaders.train):
        optimizer.zero_grad()
        logits = model(x.cuda())
        loss = loss_fn(logits, y.view(-1).long().cuda())
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    train_loss /= len(dataloaders.train)
    print(f"epoch {epoch + 1} loss = {train_loss}")


epoch 1 loss = 1.49372769812435

CPU times: user 17min 59s, sys: 4min 7s, total: 22min 7s
Wall time: 22min 32s


In [ ]:
evaluate(model)


loss = 1.4698040916998298
accuracy = 46.475372314453125
precision_macro = 18.82851219177246
recall_macro = 11.221884727478027
f1_macro = 10.278396606445312


In [ ]:
torch.save(model.state_dict(), "backprop_model_bigger_state_dict.pt")